In [8]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import math
from typing import Counter
import copy
import random
data = np.load("C:\\Users\\baljyot\\Downloads\\mnist.npz")
x_train=data['x_train']
y_train=data['y_train']
x_test=data['x_test']
y_test=data['y_test']
# indices = np.where((y_train == 0) | (y_train == 1))[0]
# X = x_train[indices]
# Y = y_train[indices]
# indices = np.where((y_test == 0) | (y_test == 1))[0]
# x_test = x_test[indices]
# y_test = y_test[indices]

# indices_0 = np.where((Y == 0))[0]
# indices_1 = np.where((Y == 1))[0]


# x_train=X[]
# y_train=Y[1000:]
# x_val=X[:1000]
# y_val=Y[:1000]

indices_0 = np.where(y_train == 0)[0]
indices_1 = np.where(y_train == 1)[0]

val_indices_0 = indices_0[:1000]
val_indices_1 = indices_1[:1000]

train_indices_0 = indices_0[1000:]
train_indices_1 = indices_1[1000:]

total_1=len(train_indices_1)
total_n1=len(train_indices_0)

train_indices = np.concatenate([train_indices_0, train_indices_1])

val_indices = np.concatenate([val_indices_0, val_indices_1])

np.random.shuffle(train_indices)
np.random.shuffle(val_indices)

# Create train and val sets
x_train = np.array(x_train[train_indices])
y_train = np.array(y_train[train_indices])
x_val = np.array(x_train[val_indices])
y_val = np.array(y_train[val_indices])

indices = np.where((y_test == 0) | (y_test == 1))[0]
x_test =np.array(x_test[indices])
y_test = np.array(y_test[indices])


flattened=[]
for i in range(len(x_train)):
    flattened.append(x_train[i].flatten())
x_train=np.array(flattened)
flattened=[]
for i in range(len(x_val)):
    flattened.append(x_val[i].flatten())
x_val=np.array(flattened)
flattened=[]
for i in range(len(x_test)):
    flattened.append(x_test[i].flatten())
x_test=np.array(flattened)


x_train=np.transpose(x_train)
sums=[]
centralisedMean=np.mean(x_train,axis=1)
centralisedData=[]
for i in range(x_train.shape[0]):
    l=[]
    for j in range(x_train.shape[1]):
        l.append(x_train[i][j]- centralisedMean[i])
    centralisedData.append(l)
centralisedMean=np.array(np.mean(centralisedData,axis=1))

S=np.matmul( centralisedData ,np.transpose(centralisedData))/18622

eigenvalues, eigenvectors = np.linalg.eigh(S)
eigenvalues = eigenvalues[::-1]
U = np.flip(eigenvectors, axis=1)

nUp=U[:,:5]

x_proj=np.matmul(nUp.T,x_train-x_train.mean(axis=1)[:,np.newaxis])

x_test=np.transpose(x_test)
x_proj_test=np.matmul(nUp.T,x_test-x_train.mean(axis=1)[:,np.newaxis])

x_val=np.transpose(x_val)
x_val_proj=np.matmul(nUp.T,x_val-x_train.mean(axis=1)[:,np.newaxis])


x_val=x_val_proj
x_test=x_proj_test
x_train=x_proj


y_train = np.array(y_train).astype(np.int8)
y_val = np.array(y_val).astype(np.int8)
y_test = np.array(y_test).astype(np.int8)

y_train[y_train == 0] = -1
y_val[y_val == 0] = -1
y_test[y_test == 0] = -1



In [16]:
weights=np.array([1]*(x_train.shape[1]))
weights.shape

(10665,)

In [21]:
total_n1+total_1

10665

In [41]:
class Region:
   def __init__(self):
      self.one=0
      self.n_one=0
      self.tot_weight=None
      self.misclassified_weight=None
      self.category=None

   def classify(self):
      if (self.one>=self.n_one): self.category=1
      else: self.category=-1

   def make_region_classify(self,one,n_one):
      self.one=one
      self.n_one=n_one
      self.classify()


   def calc_loss(self, weights, x_train, y_train):
      self.misclassified_weight=0
      self.tot_weight=0
      for i in range(len(y_train)):
         self.tot_weight+=weights[i]
         if (y_train[i]!=self.category):
            self.misclassified_weight+=weights[i]
      # print(self.misclassified_weight)

   def remove(self, to_remove, index, y_train, weights):
      remove_class = y_train[index]
      self.tot_weight -= weights[index]
      if remove_class != self.category:
         self.misclassified_weight -= weights[index]
      if remove_class == 1:
         self.one -= 1
      else:
         self.n_one -= 1
      prev_cat = self.category
      self.classify()
      if prev_cat != self.category:
         print("category changed in removing")
         self.misclassified_weight = self.tot_weight - self.misclassified_weight
   
   def add(self, to_add, index, y_train , weights):
      add_class=y_train[index]
      if (add_class==1):self.one+=1
      else: self.n_one+=1
      prev_cat=self.category
      self.classify()
      if (prev_cat!=self.category):
         print("category changed in adding")
         self.misclassified_weight = self.tot_weight - self.misclassified_weight
      self.tot_weight+=weights[index]
      if (add_class!=self.category):
          self.misclassified_weight =  self.misclassified_weight+weights[index]






   def print_info(self):
      print("one ",self.one,end="  ")
      print("n_one ",self.n_one,end="  ")
      print("class ", self.category,end="  ")

      print("misclassified ",self.misclassified_weight,end="  ")
      print("tot weight ",self.tot_weight)




In [46]:

weights=np.array([1]*(x_train.shape[1]))
total_weight=np.sum(weights)
for i in range(1):
    #all the values are unique so no need to do np.unique() (maybe beacuse of PCA)

    curr=x_train[i]
    sorted_indices = np.argsort(curr)
    s_vals =curr[sorted_indices]
    s_y_train = y_train[sorted_indices]
    # s_vals=np.unique(np.array(sorted(x_train[i])))
    print((s_vals).shape)
    print(s_vals)
    cut1=(s_vals[0]+s_vals[1])/2
    print(cut1)

    i_fr=np.where(s_vals==s_vals[0])[0]
    print(i_fr)
    i_fr_n1=0
    i_fr_1=0
    for i in i_fr:
        if (s_y_train[i]==1): i_fr_1+=1
        else: i_fr_n1+=1
    Left=Region()
    Left.make_region_classify(i_fr_1,i_fr_n1)
    

    j_fr_1=total_1-i_fr_1
    j_fr_n1=total_n1-i_fr_n1
    Right=Region()
    Right.make_region_classify(j_fr_1, j_fr_n1)

    print("Left")
    Left.print_info()
    print("Right")
    Right.print_info()

    Left.calc_loss(weights[:1],s_vals[:1],s_y_train[:1])


    Right.calc_loss(weights[1:],s_vals[1:],s_y_train[1:])
    print("Left")
    Left.print_info()
    print("Right")
    Right.print_info()
    mini_loss=Left.misclassified_weight+Right.misclassified_weight
    print(mini_loss)
    for j in range(1,len(s_vals)-2):
        cut=(s_vals[i]+s_vals[i+1])/2

        Right.remove(s_vals[i],i , s_y_train,weights)
        Left.add(s_vals[i], i , s_y_train , weights)

        print("Left")
        Left.print_info()
        print("Right")
        Right.print_info()
        new_loss=Left.misclassified_weight+Right.misclassified_weight
        print(new_loss)
        mini_loss=min(mini_loss, new_loss)


        print()

        



    # left_region=Region()
    # middle_region=Region()
    # right_region=Region()

(10665,)
[-2330.82976086 -2327.20344266 -2290.15016654 ...  1128.97522928
  1138.47192104  1139.9026577 ]
-2329.0166017598003
[0]
Left
one  0  n_one  1  class  -1  misclassified  None  tot weight  None
Right
one  5742  n_one  4922  class  1  misclassified  None  tot weight  None
Left
one  0  n_one  1  class  -1  misclassified  0  tot weight  1
Right
one  5742  n_one  4922  class  1  misclassified  4922  tot weight  10664
4922


In [46]:

s_y_train


array([-1, -1, -1, ...,  1,  1,  1], dtype=int8)